In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)


# 构建改进的CNN模型
# 卷积神经网络
model = Sequential(
    [
        # 卷积层
        
        # 连续做两次 两个卷积层相同的操作 可以帮助模型更好地提取和学习特征
        Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1), padding="same"),
        BatchNormalization(),

        Conv2D(32, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        
        MaxPooling2D((2, 2)), # 下采样，减少特征图尺寸 参数为池化窗口大小
        Dropout(0.25), # 平衡正则化和模型性能
        
        Conv2D(64, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        
        MaxPooling2D((2, 2)), # 下采样，减少特征图尺寸
        Dropout(0.25), # 平衡正则化和模型性能
        
        Conv2D(128, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        
        MaxPooling2D((2, 2)), # 下采样，减少特征图尺寸
        Dropout(0.4), # 平衡正则化和模型性能
        
        Flatten(), # -----------------------------------
        
        # 全连接神经网络
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.5), # 平衡正则化和模型性能

        Dense(10, activation="softmax"),
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # 学习率
    loss="categorical_crossentropy", # 损失函数
    metrics=["accuracy"],
)

# 回调函数
# 在损失函数平稳时减小学习率
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

# 提前停止 达到最优值时停止训练
early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)


# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=200,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_optimized_v1.keras")